In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import folium

%load_ext autotime

time: 4.4 ms (started: 2021-12-04 14:36:03 -05:00)


<h1 align="center">An Analysis of Worldwide Air Quality</h1>

## Problem Statement

Our goal of the project was the answer the following question: 

- Is there a correlation between population size and air quality?
- Additionally, is there a correlation between elevation and air quality?
- Finally, how does each of the three pollutants correlate with air quality?

## Data

To answer our questions, we used data from two APIs: Air Quality Programmatic API and the Spott REST API. 

The Air Quality Programmatic API provides city level data about air quality index and individual pollutants (PM2.5, PM10, Ozone (O3)). The API also provides data on city latitude and longitude. 

The Spott API provides city level data on population and elevation through directly searching for the city's name. 

### Air Quality Programmatic API

First, we webscraped data, using Beautiful Soup, to gather the 1000 most populated cities in the world. The website we scraped data from is https://data.mongabay.com/cities_pop_01.htm'.  

Once we webscrapped the 1000 most populated cities, we ran the list of cities through the Air Quality Programmatic API. After dropping N/A data, we stored the data in a csv format. There are 526 rows and 9 columns. The size of the stored data is 35 KB.

In [2]:
air_quality_api = pd.read_csv('waqi_df.csv')
air_quality_api.sample(5)

,Unnamed: 0,cities,aqi,latitude,longitude,pm25,pm10,o3,uvi,date
299,504,Yiwu,95,29.306841,120.075058,95.0,28.0,1.0,1.0,11-30-2021
77,102,Jeddah,147,21.543486,39.172990,147.0,82.0,13.0,1.0,11-30-2021
410,735,Linqing,102,36.811944,115.733056,102.0,28.0,3.0,1.0,11-30-2021
307,516,Pingdingshan,85,33.766190,113.192814,85.0,19.0,9.0,1.0,11-30-2021
296,501,Yulin,55,22.616400,110.143300,55.0,28.0,13.0,2.0,11-30-2021


time: 44.3 ms (started: 2021-12-04 14:36:07 -05:00)


### Spott API

For the Spott API, are main contraint was the number of free requests we received. Per account, Spott only granted us 1000 requests. To limit the number of requests, we made a list of cities from the Air Quality API and ran those cities through the Spott API.

We stored the data in a csv format. There are 526 rows and 3 columns. The size of the stored data is 14 KB.

In [3]:
spott_api = pd.read_csv('spott_df.csv')
spott_api.sample(5)

,Unnamed: 0,Cities,Population,Elevation
236,236,Tianshui,3500000.0,1153.0
110,110,Denver,682545.0,1609.0
445,445,Fuxin,689050.0,142.0
61,61,Detroit,677116.0,183.0
327,327,Saratov,863725.0,72.0


time: 14.8 ms (started: 2021-12-04 14:36:08 -05:00)


## Data Cleaning and Manipulation - Julia

In [4]:
import functions

time: 5.07 ms (started: 2021-12-04 14:36:41 -05:00)


## Time Complexity - Together

## Analysis - Analese

## Take-aways

The factor that by far had the highest correlation with Air Quality Index was the measure of the PM25 pollutant. The relationship is strong and appears to be significant. Further testing will more thoroughly analyze the significance of the relationship between PM25 levels and Air Quality Index levels. Another route we could take in the future is to gather data from cities other than the 1000 most populated cities in the world. We wonder how the correlations we analyzed would differ from medium-sized cities and small towns.

## References